# EQUITY AMERICAN OPTIONS

Valuation and Risk of American-style options on equity

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.utils import *
from financepy.market.curves import *
from financepy.products.equity import *
from financepy.models.black_scholes import *
from financepy.models.equity_lsmc import FIT_TYPES

####################################################################
# FINANCEPY BETA Version 0.33 - This build:  11 Nov 2023 at 08:55 #
#     This software is distributed FREE AND WITHOUT ANY WARRANTY   #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



# Define the Call and Put Options

In [3]:
valuation_date = Date(1, 1, 2015)

In [4]:
expiry_date = valuation_date.add_years(0.5)

In [5]:
expiry_date

01-JUL-2015

In [6]:
strike_price = 50.0

In [7]:
EU_CALL = OptionTypes.EUROPEAN_CALL
EU_PUT = OptionTypes.EUROPEAN_PUT

Let's create a put and a call using the Vanilla class

In [8]:
europeanCallOption = EquityVanillaOption(expiry_date, strike_price, EU_CALL)

In [9]:
europeanPutOption = EquityVanillaOption(expiry_date, strike_price, EU_PUT)

Now let's create a put and a call American and European option types using the American class

In [10]:
europeanAmericanCallOption = EquityAmericanOption(expiry_date, strike_price, EU_CALL)

In [11]:
europeanAmericanPutOption = EquityAmericanOption(expiry_date, strike_price, EU_PUT)

In [12]:
americanCallOption = EquityAmericanOption(expiry_date, strike_price, OptionTypes.AMERICAN_CALL)

In [13]:
americanPutOption = EquityAmericanOption(expiry_date, strike_price, OptionTypes.AMERICAN_PUT)

In [14]:
print(americanCallOption)

OBJECT TYPE: EquityAmericanOption
EXPIRY DATE: 01-JUL-2015
STRIKE PRICE: 50.0
OPTION TYPE: OptionTypes.AMERICAN_CALL
NUMBER: 1.0


In [15]:
print(americanPutOption)

OBJECT TYPE: EquityAmericanOption
EXPIRY DATE: 01-JUL-2015
STRIKE PRICE: 50.0
OPTION TYPE: OptionTypes.AMERICAN_PUT
NUMBER: 1.0


## Valuation

In [16]:
stock_price = 50.0
volatility = 0.20
interest_rate = 0.05
dividend_yield = 0.0

In [17]:
discount_curve = DiscountCurveFlat(valuation_date, interest_rate)

In [18]:
dividend_curve = DiscountCurveFlat(valuation_date, dividend_yield)

#### Analytic Model

In [19]:
model = BlackScholes(volatility)

In [20]:
europeanCallOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

3.4276581469416914

In [21]:
europeanAmericanCallOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

3.4276581469416914

#### Finite Difference Model

In [22]:
params = {
    'num_samples': 2000,
    'theta': 0.5
}
model = BlackScholes(volatility,
                     implementationType=BlackScholesTypes.FINITE_DIFFERENCE,
                     params=params)

In [23]:
europeanCallOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

3.4276581469416914

In [24]:
europeanAmericanCallOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

3.4276540933597364

In [25]:
americanCallOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

3.4276540933597364

In [26]:
europeanPutOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

2.2031750852278296

In [27]:
europeanAmericanPutOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

2.203171031583638

In [28]:
americanPutOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

2.3199039052498955

#### PSOR Model

In [29]:
params = {
    'theta': 0.5,
    'num_samples': 2000,
}
model = BlackScholes(volatility,
                     implementationType=BlackScholesTypes.PSOR,
                     params=params)

In [30]:
europeanCallOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

3.4276581469416914

In [31]:
europeanAmericanCallOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

3.4276542220179453

In [32]:
americanCallOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

3.4276542239744794

In [33]:
europeanPutOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

2.2031750852278296

In [34]:
europeanAmericanPutOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

2.203170641109721

In [35]:
americanPutOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

2.3197617864943965

#### LSMC Model

In [36]:
model = BlackScholes(volatility,
                     num_steps_per_year=52,
                     num_paths=50_000,
                     implementationType=BlackScholesTypes.LSMC,
                     params={"fit_type_value": FIT_TYPES.LAGUERRE.value,
                             "poly_degree": 3,
                             })

In [37]:
europeanCallOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

3.4276581469416914

In [38]:
europeanAmericanCallOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

3.370852795803322

In [39]:
americanCallOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

TypeError: financepy.models.equity_lsmc.equity_lsmc() got multiple values for keyword argument 'fit_type_value'

In [ ]:
europeanPutOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

2.2031750852278216

In [ ]:
europeanAmericanPutOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

2.1463697340894656

In [ ]:
americanPutOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

2.210086160210043

#### CRR Tree Model
These prices should be the same. If we insist on using the Tree the price changes a bit

In [ ]:
model = BlackScholes(volatility, BlackScholesTypes.CRR_TREE)

In [ ]:
europeanCallOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

3.4276581469416763

In [ ]:
europeanAmericanCallOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

3.427145909428062

In [ ]:
americanCallOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

3.427145909428062

In [ ]:
europeanPutOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

2.2031750852278216

In [ ]:
europeanAmericanPutOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

2.2026628477140777

These prices should be the same even though one is using the BS formula and the other uses the CRR tree

In [ ]:
americanPutOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

2.3227588252294726

This price is higher as there is an advantage to exercising early when holding an American put

## Risk Measures

You can calculate the risk measures using using perturbatory analysis

In [ ]:
americanCallOption.delta(valuation_date, stock_price, discount_curve, dividend_curve, model)

0.6240485077846358

In [ ]:
americanCallOption.gamma(valuation_date, stock_price, discount_curve, dividend_curve, model)

534.1064689190489

In [ ]:
americanCallOption.theta(valuation_date, stock_price, discount_curve, dividend_curve, model)

-4.0724784861637575

In [ ]:
americanCallOption.rho(valuation_date, stock_price, discount_curve, dividend_curve, model)

13.096682231981127

Copyright (c) 2020 Dominic O'Kane